# Data processing Pipeline <a class="anchor" id="top"></a>

Pipeline in this context is a name for a data processing pipeline which transforms data using a series of components called transforms(annotators/filters)



The list of the components used in this pipeline are:

- [Ingest2parquet](#item1)
- [Exact Dedup](#item2)
- [Doc_ID generation](#item3)
- [Fuzzy Dedup](#item4)
- [Programming Language Annotation](#item5)
- [Code quality annotation](#item6)
- [Filtering](#item7)
- [Tokenization](#item8)

The notebook executes the above transforms one after another to give a sense of
how data processing can be done using data processing library.

It is possible to change input/output paths and supply your own data and execute the pipeline. Each transform takes in a set of parameters which can also be tweaked as per requirement.

### Import Common python modules

In [ ]:

import os
import sys

from data_processing.ray import TransformLauncher
from data_processing.utils import ParamsUtils

## Sample Data For the pipeline

This sample pipeline can be used to process data needed to train/finetune models for code. Hence the data we need before starting the 
pipeline can be code repositories from say github/gitlab/bitbucket etc.

### Collecting Data

Easiest way to get data is to download repositories from github in zip format. 

Here's how to download a GitHub repository in ZIP format:

1. Go to the desired repository on GitHub.
2. Click the "Code" button near the top right corner of the repository.
3. Click the "Download ZIP" button.

This will download a ZIP archive of the entire repository to your computer.

Follow these steps and download some repositories from github into a folder. Now your data is ready.

The folder containing this data would serve as the input to the pipeline. Assign the path of this data folder to the variable `zip_input_folder`. It would be used


### Set input/output path variables for the pipeline

In [ ]:
# Example
# We can set input paths here
zip_input_folder = <UPDATE PATH TO INPUT DATA HERE>


if not os.path.exists(zip_input_folder):
    print ("NO INPUT DATA")
    print ("Please set `zip_input_folder` variable to path containing data")

# make sure the paths are correct
data_base_path = "test-data"

parquet_data_output = os.path.join(data_base_path, "parquet_input")

ededup_out =  os.path.join(data_base_path, "ededup_out")

doc_id_out =  os.path.join(data_base_path, "doc_id_out")
fdedup_out = os.path.join(data_base_path, "fdedup_out")

lang_out =  os.path.join(data_base_path,"lang_out")
cq_out = os.path.join(data_base_path,"cq_out")

filter_out = os.path.join(data_base_path ,"filter_out")
tokensization_out = os.path.join(data_base_path ,"tokenization_out")



## <span style="color: green"> 1. Convert data to parquet using ingest2parquet [<-](#top)<a class="anchor" id="item1"></a>
_zip_ to _parquet_ </span>

Raw code data files which are in zip format are converted to parquet files, where each row of the parquet file corresponds to a separate code file. Apart from the contents of the code file, every row also contains a unique document id, file URL, name of the repository, source of the data, date of acquisition and license of the repository. For every code file, a language field is also added, which is detected using the filename
extensions.

It is advised to prepare a dataset in the folder `test-data/input`. The dataset should contain zip
files of github repos. One way to make this dataset is to download github repos in zip format.


### Set Input/output Folder

In [ ]:
# For this stage input folder contains the zip files, each zip file contains a github repo.

input_folder = zip_input_folder
output_folder =  parquet_data_output

### Execute 

In [ ]:

from data_processing.utils import ParamsUtils
import sys
import ast
from ingest2parquet import ingest2parquet

# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

params = {
        "data_local_config": ParamsUtils.convert_to_ast(local_conf),
        "data_files_to_use": ast.literal_eval("['.zip']"),
        "detect_programming_lang": True,
        "snapshot": "github",
        "domain": "code"
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# Launch

ingest2parquet()

##  <span style="color: green">   2. Exact Dedup [<-](#top)<a class="anchor" id="item2"></a> </span>

Remove documents having identical code to remove bias in the training data. On the content of each document, a SHA256 hash is computed,
followed by de-duplication of record having identical hashes.

### Set Input/output Folder

In [ ]:
## For this stage the input is the folder containing parquet data which is output from the ingest2parquet tool

input_folder = parquet_data_output
output_folder = ededup_out

### Execute 

In [ ]:
# Import ededup transform configuration
from ededup_transform import EdedupTableTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus": 0.8}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": 3,
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

# create launcher
ededup_launcher = TransformLauncher(transform_runtime_config=EdedupTableTransformConfiguration())
ededup_launcher.launch()

## <span style="color: green">  3. DOC ID generation [<-](#top)<a class="anchor" id="item3"></a> </span>

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set hash_column to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set int_id_column to the name of the column, where you want to store it. **This is needed is we want to run fuzzy dedup** in the pipeline.

In [ ]:
# Input for this stage is the output of exact dedeup component
# output of this component makes it possible for fdedup component to run on data.

input_folder = ededup_out
output_folder = doc_id_out


In [ ]:
from doc_id_transform import DocIDTransformConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus": 0.8}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": 3,
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "hash_column",
    "doc_id_int_column": "int_id_column",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = TransformLauncher(transform_runtime_config=DocIDTransformConfiguration())
launcher.launch()


## 4. <span style="color: green">  Fuzzy Dedup [<-](#top)<a class="anchor" id="item4"></a> </span>

Post exact deduplication, fuzzy deduplication is applied with
the goal of removing code files that may have slight variations and thereby unbiasing
the data further. Small variations are quite commonly seen in code data in the form
of variations in the values of variables, addittion of logging statements etc. Find near-
duplicate.

### Set Input/output Folder

In [ ]:
## Input to this component is the output of doc_id generator component. 

input_folder = doc_id_out
output_folder = fdedup_out

### Execute 

In [ ]:
import os
import sys

from data_processing.ray import TransformLauncher
from data_processing.utils import ParamsUtils
from fdedup_transform import FdedupTableTransformConfiguration

# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus": 0.8}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": 3,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "int_id_column",
    "fdedup_cluster_column": "hash_column",
    # infrastructure
    "fdedup_bucket_cpu": 0.5,
    "fdedup_doc_cpu": 0.5,
    "fdedup_mhash_cpu": 0.5,
    "fdedup_num_doc_actors": 2,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 2,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.8,
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

fdedup_launcher = TransformLauncher(transform_runtime_config=FdedupTableTransformConfiguration())
fdedup_launcher.launch()

## <span style="color: green">  5. Programming language annotation [<-](#top)<a class="anchor" id="item5"></a> </span>

The raw data may contains many programming languages. Of this, we would wish to retain a prioritised list of selected programming languages. This component takes a file which has new line separated names of languages we need to select. It annotates the data a new column with boolean values. This column can be used by filter component to select the required languages.

### Set Input/output Folder

In [ ]:

input_folder = fdedup_out
output_folder = lang_out 
selected_languages_file = "./test-data/allowed-code-languages.txt"


### Execute 

In [ ]:
import os
import sys

from data_processing.ray import TransformLauncher
from data_processing.utils import ParamsUtils
from proglang_select_transform import (
    ProgLangSelectTransformConfiguration,
    lang_allowed_langs_file_key,
    lang_lang_column_key,
    lang_output_column_key,
)

# create parameters
language_column_name = "programming_language"
annotated_column_name = "lang_selected"

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus": 0.8}
langselect_config = {
    lang_allowed_langs_file_key: selected_languages_file,
    lang_lang_column_key: language_column_name,
    lang_output_column_key: annotated_column_name,
}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": 1,
    # language selection specific parameters
    **langselect_config,
}

sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = TransformLauncher(transform_runtime_config=ProgLangSelectTransformConfiguration())
launcher.launch()


## <span style="color: green">  6. Code Quality [<-](#top)<a class="anchor" id="item6"></a> </span>

We experiment with various code quality metrics but finally retain
the four code quality metrics used by (Li et al., 2023) to balance the tradeoff between
code quality versus data volume. 


### Set Input/output Folder

In [ ]:
input_folder = lang_out
output_folder = cq_out

### Execute 

In [ ]:
import os
import sys
from pathlib import Path

from code_quality_transform import CodeQualityTransformConfiguration
from data_processing.ray import TransformLauncher
from data_processing.utils import ParamsUtils

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

language_column_name = "programming_language"

worker_options = {"num_cpus": 0.8}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": 3,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_creation_delay": 0,
    "runtime_code_location": ParamsUtils.convert_to_ast(code_location),
    # code quality configuration
    "cq_contents_column_name": "contents",
    "cq_language_column_name": language_column_name,
}


Path(output_folder).mkdir(parents=True, exist_ok=True)

sys.argv = ParamsUtils.dict_to_req(d=params)

# launch
# create launcher
launcher = TransformLauncher(transform_runtime_config=CodeQualityTransformConfiguration())


cq_launcher.launch()

## 7. <span style="color: green">   Filtering [<-](#top)<a class="anchor" id="item7"></a> </span>

Filter out documents that do not meet the quality threshold for each annotation. The thresholds are computed based on a distributional
analysis as well as manual inspection of samples maintaining the balance between data quality and data volume

### Set Input/output Folder

In [ ]:
input_folder = cq_out
output_folder = filter_out

### Execute 

In [ ]:
import os

from data_processing.data_access import DataAccessLocal
from filter_transform import (
    FilterTransformConfiguration,
    filter_columns_to_drop_cli_param,
    filter_criteria_cli_param,
    filter_logical_operator_cli_param,
)

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

# This is just an example criteria to filter
filter_criteria = [
    "total_num_lines > 10 AND total_num_lines < 90",
    "lang_selected = 1",
]
filter_logical_operator = "AND"
filter_columns_to_drop = ["lang_selected", "hash_column"]

filter_params = {
    filter_criteria_cli_param: filter_criteria,
    filter_columns_to_drop_cli_param: filter_columns_to_drop,
    filter_logical_operator_cli_param: filter_logical_operator,
}

worker_options = {"num_cpus": 0.8}
launcher_params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": 5,
}


sys.argv = ParamsUtils.dict_to_req(launcher_params | filter_params)
    # Create the longer to launch with the blocklist transform.
launcher = TransformLauncher(transform_runtime_config=FilterTransformConfiguration())
    # Launch the ray actor(s) to process the input
launcher.launch()

## 8. <span style="color: green">  Tokenization [<-](#top)<a class="anchor" id="item8"></a> </span>

The data tokenization transform maps a (non-empty) input table to an output table using a pre-trained tokenizer. The input table must contain at least two columns, by default named document_id and contents. The tokenization transform utilizes the pre-trained tokenizer to tokenize each row (assuming a document) in the input table to each row in the output folder.

A pre-trained tokenizer must be specified through the --tkn_tokenizer parameter, which can be the name of a ready-for-download tokenizer from HuggingFace such as hf-internal-testing/llama-tokenizer, bigcode/starcoder or any others that can loaded by the Huggingface AutoTokenizer library. 


In [ ]:
input_folder = filter_out
output_folder = tokensization_out

In [ ]:
from tokenization_transform import TokenizationTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus": 0.8}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": 5,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = TransformLauncher(transform_runtime_config=TokenizationTransformConfiguration())
# Launch the ray actor(s) to process the input
launcher.launch()
